______
<img src="./imgs/DRRlogo.jpg" width="350" />

# Land Cover Classification for use in the CAPRA Model 

## Linear filters
_____

It is common to apply filters to enhance images for visualization or as a previous step for image processing. Linear filtering (or convolution) refers to a linear combination of pixel values in a neighborhood. The neighborhoood is specified by a  kernel, the weights of the kernel determine the coefficients in the linear combination (in GEE the term *filter* is used interchangeably with *kernel*).

Convolution filters calculate a new image applying a moving window over the original pixels contained in the defined window. The following image adapted from [Richards, J. (2013)](http://www.springer.com/gp/book/9783642300615), illustrates the operation of a moving window.<br>

<img src="./imgs/filter_window.png">

Filtering an image can be useful for extracting image information at different spatial frequencies. For this reason, smoothing filters are called low-pass filters (they let low-frequency data pass through) and edge detection filters are called high-pass filters. To implement filtering in GEE use image.convolve() with ee.kernel for the argument.

### Smoothing

Smoothing means to convolve an image with a smoothing or low-pass kernel. GEE includes several filters or kernels. A simple smoothing kernel evaluates a particular input pixel value, and the pixels sorrounding the input pixel, and outputs a new pixel value, that is the mean of this convolution. This simple filter is a square kernel with uniform weights. Convolving with this kernel sets each pixel to the mean of its neighborhood (sometimes called a "pillbox" or "boxcar" filter).

In this example, we will select a square kernel that has the following parameters: <br>
+ *radius (Float)* :  The radius of the kernel to generate. For a 3x3 kernel, we should use 1, for 5x5 kernel we should use 2 and so on. <br>
+ *units (String) *:The system of measurement for the kernel "meters" or "pixels". Default value: "pixels".<br>
+ *normalize (Boolean)*: Indicates if the kernel values should be normalized to sum 1. Default value: "True".<br>
+ *magnitude (Float) *: Scale each value by this amount. Default: 1.<br>
The kernel definition of a mean filter looks like the image below: <br> 
<img src="./imgs/filter_mean.png">
This filter will smooth the image as can be seen after applying the following code.

In [1]:
import ee
from IPython.display import Image
ee.Initialize()

# load the image collection and filter
l8sr = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')

CBD = ee.FeatureCollection("USDOS/LSIB/2013")
boundary = CBD.filterMetadata('name', 'equals', 'DOMINICAN REPUBLIC')

visimage = ee.Image(l8sr
            .filterDate('2019-01-01', '2019-04-30')
            .filterBounds(boundary)
            .sort('CLOUD_COVER')   
            .select('B[2-5]')   
            .first()
)

In [2]:
# defining the filter
lpk = ee.Kernel.square(radius = 1, units = 'pixels', normalize= True)  # filter mean of 3*3 
# apply the filter using the function convolve
smooth3x3 =    visimage.convolve(lpk)
# visualize the result
Image(url=smooth3x3.getThumbUrl({'min': 0, 'max': 2000,'bands': 'B4,B3,B2'}))

In [3]:
# Increasing the size of the moving window. 
# defining the filter
lpk = ee.Kernel.square(radius = 3, units = 'pixels', normalize= True)  # 7*7 filter 
# apply the filter using the function convolve
smooth7x7 =    visimage.convolve(lpk)
# visualize the result
Image(url=smooth7x7.getThumbUrl({'min': 0, 'max': 2048,'bands': 'B4,B3,B2'}))

A Gaussian kernel can also be used for smoothing. With a Gaussian smoothing filter more weight is given at the central pixels and less weight to the neighbors. The farther away the neighbors, the smaller the wight. Think of filtering with a Gaussian kernel as computing the weighted average 

In [5]:
import pprint
# Lets try another example, using a gaussian filter
# define the kernel, the code below will produce a discrete aproximation of a gaussian filter with sigma= 1 and size=5x5 <br> 
gauskernel = ee.Kernel.gaussian(2,1)
pprint.pprint(gauskernel.getInfo())
# apply the filter
gauss=  visimage.convolve(gauskernel)
# visualize results 
Image(url=gauss.getThumbUrl({'min': 0, 'max': 2048,'bands': 'B4,B3,B2'}))

{'center': [2, 2],
 'radius': 2.0,
 'type': 'Kernel.gaussian',
 'weights': '\n'
            '  [0.0029690167439504977, 0.013306209891013656, '
            '0.021938231279714653, 0.013306209891013656, '
            '0.0029690167439504977]\n'
            '  [0.013306209891013656, 0.05963429543618016, 0.0983203313488458, '
            '0.05963429543618016, 0.013306209891013656]\n'
            '  [0.021938231279714653, 0.0983203313488458, 0.1621028216371267, '
            '0.0983203313488458, 0.021938231279714653]\n'
            '  [0.013306209891013656, 0.05963429543618016, 0.0983203313488458, '
            '0.05963429543618016, 0.013306209891013656]\n'
            '  [0.0029690167439504977, 0.013306209891013656, '
            '0.021938231279714653, 0.013306209891013656, '
            '0.0029690167439504977]'}


### Edge detection


Convolving with an edge-detection kernel is used to find rapid changes in pixel values that usually signify edges of objects rpresented in the image data. <br>
We can also define the kernel using the <i>fixed</i> kernel option. <br>
Parameters for a fixed kernel are: <br>
+ *width (Integer)*: The width of the kernel in pixels.
+ *height (Integer)*: The height of the kernel in pixels.
+ *weights (List)*: The pixel values of the kernel.
+ *x (Integer, default: -1)*:
The location of the focus, as an offset from the left.
+ *y (Integer, default: -1)*:The location of the focus, as an offset from the top.
+ *normalize (Boolean, default: false)*:Normalize the kernel values to sum to 1.

In [6]:
# Lets apply an edge detection filter of 1x3.
# define the kernel
list =  [[-1,0 , 1]]
kernel = ee.Kernel.fixed(3, 1, list, -1, 0, False)
print(kernel.getInfo())

{'type': 'Kernel.fixed', 'width': 3, 'height': 1, 'weights': '\n  [-1.0, 0.0, 1.0]', 'x': 1, 'y': 0, 'center': [1, 0]}


In [7]:
# apply the kernel
edge1x3 = visimage.convolve(kernel)

# visualize the result
Image(url=edge1x3.getThumbUrl({'min': 0, 'max': 2000,'bands': 'B4,B3,B2'}))

In [8]:
# Lets apply a edge detection filter of 3x1.
# define the kernel
list =  [[-1],[0] , [1]]
kernel = ee.Kernel.fixed(1, 3, list, -1, 0, False)
print(kernel.getInfo())
# apply the kernel
edge1x3 = visimage.convolve(kernel)

# visualize the result
Image(url=edge1x3.getThumbUrl({'min': 0, 'max': 2048,'bands': 'B4,B3,B2'}))

{'type': 'Kernel.fixed', 'width': 1, 'height': 3, 'weights': '\n  [-1.0]\n  [0.0]\n  [1.0]', 'x': 0, 'y': 0, 'center': [0, 0]}


A classic edge detection kernel is the [Laplacian filter](https://en.wikipedia.org/wiki/Discrete_Laplace_operator).  <br>
 A possible kernel for a laplacian filter is: <br>
<img src="./imgs/laplacian.png">

In [9]:
# define the kernel for Laplacian filter
import pprint
list =  [0,1,0]
center  = [1,-4,1]
klist = [list,center,list]
kernel = ee.Kernel.fixed(3, 3, klist, -1, -1, False)

pprint.pprint(kernel.getInfo())
# apply the kernel
laplacian = visimage.convolve(kernel)

# visualize the result

Image(url=laplacian.getThumbUrl({'min': 0, 'max': 2048,'bands': 'B4,B3,B2'}))

{'center': [1, 1],
 'height': 3,
 'type': 'Kernel.fixed',
 'weights': '\n  [0.0, 1.0, 0.0]\n  [1.0, -4.0, 1.0]\n  [0.0, 1.0, 0.0]',
 'width': 3,
 'x': 1,
 'y': 1}


We have defined a particular Laplacian kernel. But, it is also possible to use some laplacian kernels that  GEE has already defined.  The following code illustrates a predefined Laplacian kernel.

In [10]:
# define the kernel <br> 
kernel = ee.Kernel.laplacian8()
pprint.pprint(kernel.getInfo())
# apply the filter
laplacian8 =  visimage.convolve(kernel)
# visualize results 
Image(url = laplacian8.getThumbUrl({'min': 0, 'max': 2048,'bands': 'B4,B3,B2'}))

{'center': [1, 1],
 'type': 'Kernel.laplacian8',
 'weights': '\n  [1.0, 1.0, 1.0]\n  [1.0, -8.0, 1.0]\n  [1.0, 1.0, 1.0]'}
